In [1]:
from config import config
import utils, json, threading, time
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/chunyuko/.cache/huggingface/token
Login successful


In [2]:
job_categories = pd.read_csv("data/20231217_job_category.csv")
job_infos = pd.read_csv("data/20231228_job_info.csv")
data = job_categories.merge(job_infos, on = "id", how = "left")
descs = [*set(data.sort_values("job_desc")["job_desc"].tolist())]

In [3]:
def fetch_content(i, result):
    
    x = """Please extract the job responsibilities from the following text and list them clearly in English bullet points.
    Make sure not to include any information that is not part of the job responsibilities.
    """ + str(i)
    
    try:
        time.sleep(0.1)
        
        content = utils.get_answer_chatgpt(x).choices[0].message.content
        
        with result_lock:
            result.append({"desc": i, "content": content})
    
    except Exception as e:
        print(f"Error processing item {i}: {e}")

result = []
result_lock = threading.Lock()
max_threads = 100

with ThreadPoolExecutor(max_workers=max_threads) as executor:
    futures = [executor.submit(fetch_content, i, result) for i in descs]

    for future in futures:
        future.result()

In [13]:
data_content = pd.merge(data.filter(["id", "job_desc"], axis = 1), pd.DataFrame(result), how = "left", left_on = "job_desc", right_on = "desc").filter(["id", "job_desc", "content"])

In [28]:
data_content = data_content[["-" in i for i in data_content["content"].tolist()]]

In [31]:
def extract_points(x):
    return [i.split("-", 1)[1].strip() for i in x.split("\n") if "-" in i]

In [34]:
data_content["content"] = data_content["content"].apply(extract_points)

/var/folders/nc/bf278_mx3rsg7d_6d9y6rc5w0000gq/T/ipykernel_80581/3597035121.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_content["content"] = data_content["content"].apply(extract_points)


In [40]:
data_content.filter(["id", "content"]).explode("content").to_csv("data/20231228_job_desc_segment.csv", index = False)